# Scrape ACS Data Using Census API

**Name: Yuran Zhu <br>
andrewID: yuranz**

This program can be used to scrape most recent ACS 5-Year Data at the block group level. It intends to scrape different tables and export as separate .csv files. It also combines tables scraped and transforms for better use.

In [1]:
import requests
import pandas as pd

## 1. Select Variables

From all available variables in ACS data, I select the following variables to scrape.<br>
Variables are divided into two types: grouped variables and selected variables.Grouped variables refer to the whole group, like `B01003`, selected variables refer to a string of detailed columns like `B01001_001E,B01001_002E,B01001_026E`. Record them in separate dictionaries.

In [2]:
# variables selection
group_var_dict = {'B01003':'population',
                  'B02001':'race',
                  'B19301':'per_capita_income',
                  'B15003':'education'} 
select_var_dict = {'B01001_001E,B01001_002E,B01001_026E':'sex',
                   'B01002_001E,B01002_002E,B01002_003E':'median_age_by_sex'}

## 2. Scrape Data

First, set up the `base_url` and `state`. Here, I use `New Jersey` as the selected state. Modify the `state` variable for changing to other states.

In [3]:
base_url = f'https://api.census.gov/data/2018/acs/acs5'
state = '34'  # modify this value for scraping other states' data
key = '7359b120f70b276a234039d6cc675abf14c0df6f'

Since the census API only support acquiring data of all block-groups for one county at one time, we need to acquire the county list of the specified state. Define the function `get_counties` to do so.

In [4]:
def get_counties(state):
    '''
    Returns the county number list for a specified state.
    '''
    county_url = f'{base_url}?get=NAME&for=county:*&in=state:{state}&key={key}'
    county_data = requests.get(county_url).json()
    df = pd.DataFrame(county_data[1:], columns = county_data[0])
    county = list(df['county'])
    return county

**i. For Grouped Data**

In [5]:
def get_group_data(state, var):
    '''
    Gets data related to one group variable, for all block-groups in a specified state.
    '''
    county = get_counties(state)
    df_lst = []
    for c in county:
        data_url = f'{base_url}?get=group({var})&for=block%20group:*&in=state:{state}%20county:{c}&key={key}'  
        data = requests.get(data_url).json()
        df = pd.DataFrame(data[1:], columns = data[0])
        drop_cols = [column for column in df if column.endswith('A') or column.endswith('M') ]
        df_cleaned = df.drop(drop_cols, axis = 1)
        df_lst.append(df_cleaned)
    df_all = pd.concat(df_lst, ignore_index = True)   
    return df_all

**ii. For Selected Data**

In [6]:
def get_select_data(state, var):
    '''
    Gets data related to one slected variables string, for all block-groups in a specified state.
    '''
    county = get_counties(state)
    df_lst = []
    for c in county:
        data_url = f'{base_url}?get={var},NAME&for=block%20group:*&in=state:{state}%20county:{c}&key={key}'  
        data = requests.get(data_url).json()
        df = pd.DataFrame(data[1:], columns = data[0])
        drop_cols = [column for column in df if column.endswith('A') or column.endswith('M') ]
        df_cleaned = df.drop(drop_cols, axis = 1)
        df_lst.append(df_cleaned)
    df_all = pd.concat(df_lst, ignore_index = True)   
    return df_all

Define functions to translate column names, making it meaningful.

In [7]:
def update_cols(df, var):
    '''
    Decodes column names with actual meaning.
    '''
    var_url = f'https://api.census.gov/data/2018/acs/acs5/groups/{var}/'
    var_lst = requests.get(var_url).json()  # scrape info for var ID and labels
    select_cols = [x for x in var_lst['variables'].keys() if x.endswith('E')]
    labels = {select_cols[x]:var_lst['variables'][select_cols[x]]['label'].split('!!')[-1]  # extract most minor group
          for x in range(len(select_cols))}
    df.rename(columns=labels, inplace = True)
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace("'s", '')
    df.columns = df.columns.str.replace(',', '')
    return df

In [8]:
def update_select_cols(df, var):
    '''
    Decodes column names with actual meaning.
    '''
    select_cols = var.split(',')
    group_var = select_cols[0].split('_')[0]  # The group the variables are assigned to
    var_url = f'https://api.census.gov/data/2018/acs/acs5/groups/{group_var}/'
    var_lst = requests.get(var_url).json()
    labels = {select_cols[x]:var_lst['variables'][select_cols[x]]['label'].split('!!')[-1]
          for x in range(len(select_cols))}
    df.rename(columns=labels, inplace = True)
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace("'s", '')
    df.columns = df.columns.str.replace(',', '')
    return df

In [9]:
def transform_group_data(state, var):
    '''
    Downloads data and tranforms it.
    '''
    df = get_group_data(state, var)
    df_trans = update_cols(df, var)
    return df_trans

In [10]:
def transform_select_data(state, var):
    '''
    Downloads data and tranforms it.
    '''
    df = get_select_data(state, var)
    df_trans = update_select_cols(df, var)
    return df_trans

## 3. Export Data

For further use, we may want to export each data set separately. Define functions `save_group_data(state, var)` and `save_select_data(state, var)` to achieve it. Later, join datasets together adn export a whole dataset.

In [11]:
def save_group_data(state, var):
    '''
    Exports grouped downloaded and tranformed data to local directory, saving as .csv.
    '''
    df = transform_group_data(state, var)
    fname = group_var_dict[var]  # match fname with data info in dictionary
    df.to_csv('./%s.csv' % (fname), index = False)
    print('./%s.csv has been exported.' % (fname))  # print message after successfully exporting
    return

def save_select_data(state, var):
    '''
    Exports selected downloaded and tranformed data to local directory, saving as .csv.
    '''
    df = transform_select_data(state, var)
    fname = select_var_dict[var]
    df.to_csv('./%s.csv' % (fname), index = False)
    print('./%s.csv has been exported.' % (fname))
    return

In [12]:
# Apply to all variables we selected in the dictionary
for var in group_var_dict.keys():
    save_group_data(state, var)
for var in select_var_dict.keys():
    save_select_data(state, var)
print("Finished!")

./population.csv has been exported.
./race.csv has been exported.
./per_capita_income.csv has been exported.
./education.csv has been exported.
./sex.csv has been exported.
./median_age_by_sex.csv has been exported.
Finished!


## 4. Transform Datasets

In [23]:
# Import datasets
df_ppl = pd.read_csv("population.csv")
df_race = pd.read_csv("race.csv")
df_inc = pd.read_csv("per_capita_income.csv")
df_edu = pd.read_csv("education.csv")
df_sex = pd.read_csv("sex.csv")
df_medage = pd.read_csv("median_age_by_sex.csv")

For `race` data, select those columns containing information of specific race, such as 'Write only'. For `education` data, select those columns refer to an Associate degree or higher.

In [24]:
# change column names and drop specific columns 
df_ppl.rename(columns={'Total':'Population'}, inplace = True)
df_race.drop(df_race.columns[8:11], axis=1, inplace=True)
df_race = df_race.drop(columns = ['Total'])
df_inc.rename(columns={df_inc.columns[1]:'Per_capita_income'}, inplace = True)
df_edu.drop(df_edu.columns[2:21], axis=1, inplace=True)
df_edu = df_edu.drop(columns = ['Total'])
df_sex = df_sex.drop(columns = ['Total'])
df_medage.rename(columns={'Total':'Median_age', 'Male':'Male median_age' ,'Female':'Female_median_age'},
                 inplace = True)

In [25]:
# merge tables and export
df_list = [df_race, df_inc, df_edu, df_sex, df_medage]
acs_df = df_ppl
for df in df_list:
    acs_df = pd.merge(acs_df, df)
acs_df = acs_df.drop(columns = ['NAME'])
acs_df.to_csv('./acs_data.csv', index = False)